### Coding Exercises

Consider the following classes:

In [80]:
class Serializer:
    def __init__(self):
        self.object = self.__class__.__name__

    def toJSON(self):
        return vars(self)

class Stock(Serializer):
    def __init__(self, symbol, date, open_, high, low, close, volume):
        super().__init__()
        self.symbol = symbol
        self.date = date
        self.open = open_
        self.high = high
        self.low = low
        self.close = close
        self.volume = volume

        
class Trade(Serializer):
    def __init__(self, symbol, timestamp, order, price, volume, commission):
        self.symbol = symbol
        self.timestamp = timestamp
        self.order = order
        self.price = price
        self.commission = commission
        self.volume = volume


#### Exercise 1

Given the above class, write a custom `JSONEncoder` class to **serialize** dictionaries that contain instances of these particular classes. Keep in mind that you will want to deserialize the data too - so you will need some technique to indicate the object type in your serialization.

For example you may have an object such as this one that needs to be serialized:

In [91]:
from datetime import date, datetime
from decimal import Decimal
import json

activity = {
    "quotes": [
        Stock('TSLA', date(2018, 11, 22), 
              Decimal('338.19'), Decimal('338.64'), Decimal('337.60'), Decimal('338.19'), 365_607),
        Stock('AAPL', date(2018, 11, 22), 
              Decimal('176.66'), Decimal('177.25'), Decimal('176.64'), Decimal('176.78'), 3_699_184),
        Stock('MSFT', date(2018, 11, 22), 
              Decimal('103.25'), Decimal('103.48'), Decimal('103.07'), Decimal('103.11'), 4_493_689)
    ],
    
    "trades": [
        Trade('TSLA', datetime(2018, 11, 22, 10, 5, 12), 'buy', Decimal('338.25'), 100, Decimal('9.99')),
        Trade('AAPL', datetime(2018, 11, 22, 10, 30, 5), 'sell', Decimal('177.01'), 20, Decimal('9.99'))
    ]
}

In [82]:
class CustomJSONEncoder(json.JSONEncoder):
    def default(self, arg):
        try:
            vars = arg.toJSON()
        except AttributeError:
            return arg
        else:
            for key, value in vars.items():
                if isinstance(value, datetime):
                    vars[key] = value.strftime('%Y-%m-%dT%H:%M:%S')
                elif isinstance(value, date):
                    vars[key] = value.strftime('%Y-%m-%d')
                elif isinstance(value, Decimal):
                    vars[key] = float(value)
            return vars

In [83]:
from pprint import pprint
ser_d = json.dumps(activity, cls=CustomJSONEncoder, indent=2)
print(ser_d)

{
  "quotes": [
    {
      "object": "Stock",
      "symbol": "TSLA",
      "date": "2018-11-22",
      "open": 338.19,
      "high": 338.64,
      "low": 337.6,
      "close": 338.19,
      "volume": 365607
    },
    {
      "object": "Stock",
      "symbol": "AAPL",
      "date": "2018-11-22",
      "open": 176.66,
      "high": 177.25,
      "low": 176.64,
      "close": 176.78,
      "volume": 3699184
    },
    {
      "object": "Stock",
      "symbol": "MSFT",
      "date": "2018-11-22",
      "open": 103.25,
      "high": 103.48,
      "low": 103.07,
      "close": 103.11,
      "volume": 4493689
    }
  ],
  "trades": [
    {
      "symbol": "TSLA",
      "timestamp": "2018-11-22T10:05:12",
      "order": "buy",
      "price": 338.25,
      "commission": 9.99,
      "volume": 100
    },
    {
      "symbol": "AAPL",
      "timestamp": "2018-11-22T10:30:05",
      "order": "sell",
      "price": 177.01,
      "commission": 9.99,
      "volume": 20
    }
  ]
}


Hint: You can modify the classes if you need to.

#### Exercise 2

Write code to reverse the serialization you just created. Write a custom decoder that can deserialize a JSON structure containing `Stock` and `Trade` objects. 

In [84]:
import re

class CustomJSONDecoder(json.JSONDecoder):
    def __init__(self, *args, **kwargs):
        super().__init__(parse_float=Decimal)
    
    def decode(self, arg):
        obj = super().decode(arg)
        date_pattern = r'^\d{4}-\d{2}-\d{2}$'
        time_part = r'\s([01]\d|2[0-3]):[0-5]\d(:[0-5]\d)?$'

        datetime_pattern = date_pattern + time_part
        if re.search(datetime_pattern, arg):
            obj = datetime(obj)
        elif re.search(date_pattern, arg):
            obj = date(obj)
        return obj

        

In [85]:
deser_d = json.loads(ser_d, cls=CustomJSONDecoder)
pprint(deser_d)

{'quotes': [{'close': Decimal('338.19'),
             'date': '2018-11-22',
             'high': Decimal('338.64'),
             'low': Decimal('337.6'),
             'object': 'Stock',
             'open': Decimal('338.19'),
             'symbol': 'TSLA',
             'volume': 365607},
            {'close': Decimal('176.78'),
             'date': '2018-11-22',
             'high': Decimal('177.25'),
             'low': Decimal('176.64'),
             'object': 'Stock',
             'open': Decimal('176.66'),
             'symbol': 'AAPL',
             'volume': 3699184},
            {'close': Decimal('103.11'),
             'date': '2018-11-22',
             'high': Decimal('103.48'),
             'low': Decimal('103.07'),
             'object': 'Stock',
             'open': Decimal('103.25'),
             'symbol': 'MSFT',
             'volume': 4493689}],
 'trades': [{'commission': Decimal('9.99'),
             'order': 'buy',
             'price': Decimal('338.25'),
             '

#### Exercise 3

Do the same serialization and deserialization, but using `Marshmallow`.

In [101]:
from marshmallow import Schema, fields, post_load

class StockSchema(Schema):
    symbol = fields.Str()
    date = fields.Date()
    open = fields.Decimal(as_string=True)
    high = fields.Decimal(as_string=True)
    low = fields.Decimal(as_string=True)
    close = fields.Decimal(as_string=True)
    volume = fields.Int()

    @post_load
    def make_stock(self, data, **kwargs):
        data['open_'] = data.pop('open')
        return Stock(**data)

        
class TradeSchema(Schema):
    symbol = fields.Str()
    timestamp = fields.DateTime()
    order = fields.Str()
    price = fields.Decimal(as_string=True)
    commission = fields.Decimal(as_string=True)
    volume = fields.Int()

    @post_load
    def make_trade(self, data, **kwargs):
        return Trade(**data)


class ActivitySchema(Schema):
    quotes = fields.Nested(StockSchema, many=True)
    trades = fields.Nested(TradeSchema, many=True)


result = ActivitySchema().dumps(activity)
activity_deser = ActivitySchema().loads(result)


    

In [98]:
result

'{"quotes": [{"symbol": "TSLA", "date": "2018-11-22", "open": "338.19", "high": "338.64", "low": "337.60", "close": "338.19", "volume": 365607}, {"symbol": "AAPL", "date": "2018-11-22", "open": "176.66", "high": "177.25", "low": "176.64", "close": "176.78", "volume": 3699184}, {"symbol": "MSFT", "date": "2018-11-22", "open": "103.25", "high": "103.48", "low": "103.07", "close": "103.11", "volume": 4493689}], "trades": [{"symbol": "TSLA", "timestamp": "2018-11-22T10:05:12", "order": "buy", "price": "338.25", "commission": "9.99", "volume": 100}, {"symbol": "AAPL", "timestamp": "2018-11-22T10:30:05", "order": "sell", "price": "177.01", "commission": "9.99", "volume": 20}]}'

In [102]:
pprint(activity_deser)

{'quotes': [<__main__.Stock object at 0x0000022A06151D50>,
 'trades': [<__main__.Trade object at 0x0000022A0593ECF0>,
            <__main__.Trade object at 0x0000022A0593EBE0>]}
